In [1]:
# Lab 10 MNIST and softmax
import torch
import torchvision.datasets as dsets
import torchvision.transforms as transforms
import random

In [2]:
# cuda를 사용할 수 있으면 cuda를 사용하고 아니면 cpu를 사용한다
device = 'cuda' if torch.cuda.is_available() else 'cpu'

# for reproducibility
random.seed(777)
torch.manual_seed(777)
if device == 'cuda':
    torch.cuda.manual_seed_all(777)

In [3]:
# parameters
learning_rate = 0.001
training_epochs = 15
batch_size = 100

In [4]:
# MNIST dataset를 불러온다
mnist_train = dsets.MNIST(root='MNIST_data/',
                          train=True,
                          transform=transforms.ToTensor(),
                          download=True)

mnist_test = dsets.MNIST(root='MNIST_data/',
                         train=False,
                         transform=transforms.ToTensor(),
                         download=True)

In [5]:
# 데이터셋으로 dataset loader를 만들어 학습에 사용한다
data_loader = torch.utils.data.DataLoader(dataset=mnist_train,
                                          batch_size=batch_size,
                                          shuffle=True,
                                          drop_last=True)

In [6]:
# nn layers를 생성한다.
linear1 = torch.nn.Linear(784, 512, bias=True)
linear2 = torch.nn.Linear(512, 512, bias=True)
linear3 = torch.nn.Linear(512, 512, bias=True)
linear4 = torch.nn.Linear(512, 512, bias=True)
linear5 = torch.nn.Linear(512, 10, bias=True)
relu = torch.nn.ReLU() # 활성화함수로 ReLU를 사용한다

In [7]:
# xavier initialization. 레이어가 받는 입력값의 개수, 출력값의 개수에 따라 가중치를 초기화한다
torch.nn.init.xavier_uniform_(linear1.weight)
torch.nn.init.xavier_uniform_(linear2.weight)
torch.nn.init.xavier_uniform_(linear3.weight)
torch.nn.init.xavier_uniform_(linear4.weight)
torch.nn.init.xavier_uniform_(linear5.weight)

Parameter containing:
tensor([[-0.0565,  0.0423, -0.0155,  ...,  0.1012,  0.0459, -0.0191],
        [ 0.0772,  0.0452, -0.0638,  ...,  0.0476, -0.0638,  0.0528],
        [ 0.0311, -0.1023, -0.0701,  ...,  0.0412, -0.1004,  0.0738],
        ...,
        [ 0.0334,  0.0187, -0.1021,  ...,  0.0280, -0.0583, -0.1018],
        [-0.0506, -0.0939, -0.0467,  ..., -0.0554, -0.0325,  0.0640],
        [-0.0183, -0.0123,  0.1025,  ..., -0.0214,  0.0220, -0.0741]],
       requires_grad=True)

In [8]:
# Sequential model을 생성한다
model = torch.nn.Sequential(linear1, relu, linear2, relu, linear3).to(device)

In [9]:
# define cost/loss & optimizer를 정의한다
criterion = torch.nn.CrossEntropyLoss().to(device)    # 다중분류에 많이 쓰이는 CrossEntropyLoss를 사용한다
optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)

In [10]:
# 모델을 학습시킨다
total_batch = len(data_loader)
for epoch in range(training_epochs):
    avg_cost = 0

    for X, Y in data_loader:
        # reshape input image into [batch_size by 784]
        # label is not one-hot encoded
        # view는 reshpae와 같은 역할을 하며 to(device)는 device에 변수를 할당한다는 뜻이다. 
        X = X.view(-1, 28 * 28).to(device)
        Y = Y.to(device)

        # 역전파
        optimizer.zero_grad()
        hypothesis = model(X)
        cost = criterion(hypothesis, Y) # CrossEntropyLoss로 loss를 구한다
        cost.backward()
        optimizer.step()

        avg_cost += cost / total_batch # 에포크당 평균 에러를 구한다

    print('Epoch:', '%04d' % (epoch + 1), 'cost =', '{:.9f}'.format(avg_cost)) # 에포크당 평균 에러를 출력한다

print('Learning finished')

Epoch: 0001 cost = 0.285587102
Epoch: 0002 cost = 0.089795619
Epoch: 0003 cost = 0.059091564
Epoch: 0004 cost = 0.040617324
Epoch: 0005 cost = 0.031854805
Epoch: 0006 cost = 0.022626026
Epoch: 0007 cost = 0.022235204
Epoch: 0008 cost = 0.019418934
Epoch: 0009 cost = 0.016276943
Epoch: 0010 cost = 0.014547653
Epoch: 0011 cost = 0.012092041
Epoch: 0012 cost = 0.013070161
Epoch: 0013 cost = 0.011171374
Epoch: 0014 cost = 0.009097889
Epoch: 0015 cost = 0.012839760
Learning finished


In [11]:
# Test the model using test sets
# 테스트
with torch.no_grad(): # no_grad()를 이용해 그레디언트 트레킹을 중단시킨다. 그러면 메모리 사용을 줄일 수 있다
    X_test = mnist_test.test_data.view(-1, 28 * 28).float().to(device)
    Y_test = mnist_test.test_labels.to(device)
    
    # 정확도를 계산한다
    prediction = model(X_test)
    correct_prediction = torch.argmax(prediction, 1) == Y_test
    accuracy = correct_prediction.float().mean()
    print('Accuracy:', accuracy.item())

    # Get one and predict
    # 데이터셋에 있는 데이터 중 랜덤하게 하나 고른다
    r = random.randint(0, len(mnist_test) - 1)
    X_single_data = mnist_test.test_data[r:r + 1].view(-1, 28 * 28).float().to(device)
    Y_single_data = mnist_test.test_labels[r:r + 1].to(device)
    
    # single_data에 대한 예측값도 계산 후 라벨 데이터와 비교한다 
    print('Label: ', Y_single_data.item())
    single_prediction = model(X_single_data)
    print('Prediction: ', torch.argmax(single_prediction, 1).item())

/Users/minkyukim/opt/anaconda3/lib/python3.8/site-packages/torchvision/datasets/mnist.py:67: UserWarning: test_data has been renamed data
  warnings.warn("test_data has been renamed data")
/Users/minkyukim/opt/anaconda3/lib/python3.8/site-packages/torchvision/datasets/mnist.py:57: UserWarning: test_labels has been renamed targets
  warnings.warn("test_labels has been renamed targets")


Accuracy: 0.9782000184059143
Label:  8
Prediction:  8
